SIDE NOTE: would love to get any critiques, hints, tips, oberservations on any level not just on ML (but obviously ML as the priority).  Beginner Python learner (though I took the Full Stack Course) so any suggestions there are welcome.  Thanks.

Will need to import these libs plus setting some global variables

In [1]:
import sys
import os
import numpy as np
import renders as rs
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from IPython.display import display # Allows the use of display() for DataFrames
from sklearn.externals import joblib
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
from sklearn.cross_validation import StratifiedKFold
from sklearn.svm import SVC
import matplotlib.cm as cm
# Show matplotlib plots inline (nicely formatted in the notebook)
%matplotlib inline

# Might need to change the path of the included libraries.
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats/match_stats.py')
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats/form_model.py')
sys.path.append('/anaconda/envs/stats/lib/python3.5/site-packages')
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats/model_libs.py')
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats/form_data.py')
sys.path.append('/Users/senzari/Machine_Learning/stats')
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats')
# print(sys.path)
os.chdir('/Users/senzari/Machine_Learning/stats/src')
#print(os.getcwd())

from stats import form_data, match_stats, model_libs, form_model, predict_matches

# Variables
round_number = 27 # for MLS only
target_col = 'points'
ignore_cols = ['match_id', 'team_id', 'team_name', 'opp_id', 'opp_name', 'scheduled', 'games_played', 'round']
sub_cols = ['current_formation', 'avg_goals_against', 'goal_diff', 'win_percentage', 'sos',
           'opp_win_percentage', 'opp_sos', 'current_team_yellow_cards', 'current_team_corner_kicks', 'current_team_first_half_goals', 'current_team_sec_half_goals', 
           'opp_team_yellow_cards', 'opp_team_corner_kicks', 'opp_team_first_half_goals', 'opp_team_sec_half_goals']

all_models = ['log', 'svc', 'gmm', 'knn', 'gnb', 'randomForest']

""" this variable 'testing' should be false if using CSV's and not pulling from the database. """
testing = False

INITIALIZED...


In [2]:
data_csv = 'raw_data_squared.csv'

if testing:
    raw_data = form_data.run_data()
    raw_data.to_csv(data_csv)
    print("Raw Data Saved to CSV")
else:
    #Reading in a CSV adds the first index column
    raw_data = pd.read_csv(data_csv)
    raw_data = raw_data.drop(raw_data.columns[[0]], axis=1)

pd.set_option("display.max_columns", 85)
print('Data Loaded...')
print("Dataset size :: {}".format(raw_data.shape))
display(raw_data.head())

Data Loaded...
Dataset size :: (634, 21)


,match_id,team_id,team_name,opp_id,opp_name,scheduled,round,games_played,is_home,current_formation,goals_for,goals_allowed,opp_goals_allowed,goal_efficiency,opp_defensive_goal_efficiency,ratio_of_attacks,opp_ratio_of_attacks,ratio_ball_safe_to_dangerous_attacks,opp_ratio_ball_safe_to_dangerous_attacks,goals,points
0,249,21,FC Dallas,33,DC United,2016-03-26 21:30:00,4,3,0,4-2-3-1,4,5,5,0.114286,0.750000,0.587097,0.524917,1.210938,1.037931,3,3
1,255,21,FC Dallas,39,Columbus Crew,2016-04-03 01:00:00,5,3,1,4-2-3-1,5,5,4,0.156250,0.818182,0.502976,0.538136,1.183099,0.874074,1,1
2,265,21,FC Dallas,32,San Jose Earthquakes,2016-04-10 01:00:00,6,3,1,4-4-2,6,1,5,0.133333,0.821429,0.495575,0.504323,1.185315,1.074303,2,1
3,272,21,FC Dallas,30,Portland Timbers,2016-04-14 02:30:00,7,3,0,4-4-2,6,3,7,0.146341,0.695652,0.467742,0.609907,1.148148,1.247104,3,3
4,282,21,FC Dallas,27,Sporting Kansas City,2016-04-17 23:00:00,7,3,1,4-4-2,6,3,2,0.146341,0.942857,0.467742,0.534946,1.148148,1.053824,2,3


## FORMATTING

Converting Goals to our binary classification (0-1) and (2+) and also converting the sos and rpi and implementing the team to a particular rpi quartile

In [14]:
# Helper Function - Removes Columns to Ignore and Splits the Target Column
def split_target(data):
    td = model_libs._clone_and_drop(data, ignore_cols)
    (y, X) = model_libs._extract_target(td, target_col)
    return X, y

""" Need to do some formatting of the Data before we run the models"""
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 1000)

rankings_data = raw_data
rankings_data["offensive_ranking"] = pd.Series(None, index=rankings_data.index)
rankings_data["defensive_ranking"] = pd.Series(None, index=rankings_data.index)

#leagues = model_libs.get_leagues_country_codes()
leagues = { "epl": 'ENG' }
teams = form_data.get_teams()
league_rounds = model_libs.get_leagues_rounds()

""" Going through each League"""
for key, value in leagues.iteritems():
    print(key)
    country_code = leagues[key]
    #round_num = league_rounds[key]
    round_num = 6
    teams_in_league = teams[teams["country_code"] == country_code]
    """ Looping through the Rounds """
    for i in range(4, round_num):
        print("ROUND :: {} ".format(i))
        offensive_rankings = form_data.get_rankings(teams_in_league, i, "offensive")
        rankings = model_libs.quartile_list(offensive_rankings, True)
        offensive_rankings["offensive_rankings_quartiled"] = rankings
        #print(offensive_rankings)
        
        defensive_rankings = form_data.get_rankings(teams_in_league, i, "defensive")
        rankings = model_libs.quartile_list(defensive_rankings, False)
        defensive_rankings["defensive_rankings_quartiled"] = rankings
        #print(defensive_rankings)
        
        """ Loop through each Team in the League for that round and assign an Offensive Rank """
        for key, team in teams_in_league.iterrows():
            
            offensive_rank = offensive_rankings.loc[offensive_rankings[0] == team['id'], "offensive_rankings_quartiled"]
            idx = rankings_data.loc[(rankings_data["team_id"] == team["id"]) 
                    & (rankings_data["round"] == (round_num-1)), "offensive_ranking"].index

            rankings_data.set_value(idx, "offensive_ranking", offensive_rank.values[0])
        
            defensive_rank = defensive_rankings.loc[defensive_rankings[0] == team['id'], "defensive_rankings_quartiled"]
            opp_idx = rankings_data.loc[(rankings_data["opp_id"] == team["id"]) 
                    & (rankings_data["round"] == (round_num-1)), "defensive_ranking"].index
            
            rankings_data.set_value(opp_idx, "defensive_ranking", defensive_rank.values[0])


new_data = rankings_data
#display(new_data.loc[new_data["round"] == 4])
                          

""" Formatting data to convert goals scored to the correct category"""
# Not using points as a target for this version, using goals
classifier_data = raw_data.drop('points', 1)

classifier_data['converted_goals'] = classifier_data.apply(lambda row: model_libs.set_group(row['goals']), axis=1)

classifier_data = classifier_data.drop(ignore_cols + ['current_formation', 'goals'], 1)
display(classifier_data.head())

epl
ROUND :: 4 
TEAM ID :: 41
TEAM ID :: 42
TEAM ID :: 43
TEAM ID :: 44
TEAM ID :: 45
TEAM ID :: 46
TEAM ID :: 47
TEAM ID :: 48
TEAM ID :: 49
TEAM ID :: 50
TEAM ID :: 51
TEAM ID :: 52
TEAM ID :: 53
TEAM ID :: 54
TEAM ID :: 55
TEAM ID :: 56
TEAM ID :: 57
TEAM ID :: 58
TEAM ID :: 59
TEAM ID :: 60
TEAM ID :: 41
TEAM ID :: 42
TEAM ID :: 43
TEAM ID :: 44
TEAM ID :: 45
TEAM ID :: 46
TEAM ID :: 47
TEAM ID :: 48
TEAM ID :: 49
TEAM ID :: 50
TEAM ID :: 51
TEAM ID :: 52
TEAM ID :: 53
TEAM ID :: 54
TEAM ID :: 55
TEAM ID :: 56
TEAM ID :: 57
TEAM ID :: 58
TEAM ID :: 59
TEAM ID :: 60
ROUND :: 5 
TEAM ID :: 41
TEAM ID :: 42
TEAM ID :: 43
TEAM ID :: 44
TEAM ID :: 45
TEAM ID :: 46
TEAM ID :: 47
TEAM ID :: 48
TEAM ID :: 49
TEAM ID :: 50
TEAM ID :: 51
TEAM ID :: 52
TEAM ID :: 53
TEAM ID :: 54
TEAM ID :: 55
TEAM ID :: 56
TEAM ID :: 57
TEAM ID :: 58
TEAM ID :: 59
TEAM ID :: 60
TEAM ID :: 41
TEAM ID :: 42
TEAM ID :: 43
TEAM ID :: 44
TEAM ID :: 45
TEAM ID :: 46
TEAM ID :: 47
TEAM ID :: 48
TEAM ID :: 49
TEAM I

,is_home,goals_for,goals_allowed,opp_goals_allowed,goal_efficiency,opp_defensive_goal_efficiency,ratio_of_attacks,opp_ratio_of_attacks,ratio_ball_safe_to_dangerous_attacks,opp_ratio_ball_safe_to_dangerous_attacks,offensive_ranking,defensive_ranking,converted_goals
0,0,4,5,5,0.114286,0.750000,0.587097,0.524917,1.210938,1.037931,NaN,NaN,1
1,1,5,5,4,0.156250,0.818182,0.502976,0.538136,1.183099,0.874074,NaN,NaN,0
2,1,6,1,5,0.133333,0.821429,0.495575,0.504323,1.185315,1.074303,NaN,NaN,1
3,0,6,3,7,0.146341,0.695652,0.467742,0.609907,1.148148,1.247104,NaN,NaN,1
4,1,6,3,2,0.146341,0.942857,0.467742,0.534946,1.148148,1.053824,NaN,NaN,1


In [12]:
display(new_data)
raw_data.to_csv('new_data.csv')

,match_id,team_id,team_name,opp_id,opp_name,scheduled,round,games_played,is_home,current_formation,goals_for,goals_allowed,opp_goals_allowed,goal_efficiency,opp_defensive_goal_efficiency,ratio_of_attacks,opp_ratio_of_attacks,ratio_ball_safe_to_dangerous_attacks,opp_ratio_ball_safe_to_dangerous_attacks,goals,points,offensive_ranking,defensive_ranking
0,249,21,FC Dallas,33,DC United,2016-03-26 21:30:00,4,3,0,4-2-3-1,4,5,5,0.114286,0.750000,0.587097,0.524917,1.210938,1.037931,3,3,NaN,NaN
1,255,21,FC Dallas,39,Columbus Crew,2016-04-03 01:00:00,5,3,1,4-2-3-1,5,5,4,0.156250,0.818182,0.502976,0.538136,1.183099,0.874074,1,1,NaN,NaN
2,265,21,FC Dallas,32,San Jose Earthquakes,2016-04-10 01:00:00,6,3,1,4-4-2,6,1,5,0.133333,0.821429,0.495575,0.504323,1.185315,1.074303,2,1,NaN,NaN
3,272,21,FC Dallas,30,Portland Timbers,2016-04-14 02:30:00,7,3,0,4-4-2,6,3,7,0.146341,0.695652,0.467742,0.609907,1.148148,1.247104,3,3,NaN,NaN
4,282,21,FC Dallas,27,Sporting Kansas City,2016-04-17 23:00:00,7,3,1,4-4-2,6,3,2,0.146341,0.942857,0.467742,0.534946,1.148148,1.053824,2,3,NaN,NaN
5,289,21,FC Dallas,28,Vancouver Whitecaps FC,2016-04-24 02:30:00,8,3,0,4-4-2,7,4,5,0.205882,0.883721,0.560000,0.459259,1.016260,0.989011,0,0,NaN,NaN
6,295,21,FC Dallas,31,NY Red Bulls,2016-04-29 23:00:00,9,3,0,4-4-2,5,5,6,0.156250,0.806452,0.606742,0.426380,1.011364,0.942197,0,0,NaN,NaN
7,307,21,FC Dallas,34,Toronto FC,2016-05-07 23:30:00,10,3,0,4-2-3-1,2,8,2,0.064516,0.928571,0.564356,0.454849,0.958861,0.993355,0,0,NaN,NaN
8,315,21,FC Dallas,30,Portland Timbers,2016-05-12 01:00:00,11,3,1,4-3-3,0,8,4,0.000000,0.914894,0.587413,0.542857,0.979452,1.076923,2,3,NaN,NaN
9,324,21,FC Dallas,37,Seattle Sounders,2016-05-15 01:00:00,11,3,1,4-3-3,0,8,3,0.000000,0.884615,0.587413,0.403390,0.979452,0.954693,2,3,NaN,NaN


### RUNNING CLASSIFICATION MODEL ON IF TEAMS SCORE 0-1 OR 1-2 ON GAMES

In [ ]:
def run_features(data, drop_data, target, models):
    
    new_data = data.drop(drop_data, axis=1)
    
    #display(new_data.head())
    
    (y, X) = model_libs._extract_target(new_data, target)
    
    models = form_model.train_models(round_number, X, y, models)
    
    return models


#### Running ALL Features 
models_test_1 = run_features(classifier_data, [], 'converted_goals', ["knn"])

(classifier_y, classifier_X) = model_libs._extract_target(classifier_data, 'converted_goals')

def check_accuracy(model, data_X):
    actual_y = pd.DataFrame(classifier_y.values, columns=['actual'])
    predictions = pd.DataFrame(model.predict(data_X), columns=['predictions'])
    preds = pd.concat([predictions, actual_y], axis=1)
    preds['diff'] = preds.apply(lambda r: model_libs.predictions_diff(r['predictions'], r['actual']), axis=1)
    accuracy = np.divide(preds['diff'].sum(), float(len(preds['diff'])))
    print(accuracy)

for m in models_test_1:
    check_accuracy(m, classifier_X)

In [ ]:
def train_tuned_models(round_num, X, y):
    for i in all_models:
        models = form_model.build_tuned_model(X, y, i)
        
    return models
        
tuned_models = train_tuned_models(round_number, classifier_X, classifier_y)

In [ ]:
random_forest_model = form_model.build_tuned_model(classifier_X, classifier_y, 'randomForest')

for m in random_forest_model:
    print(m)
    check_accuracy(m, classifier_X)

In [ ]:
knn_model = form_model.build_tuned_model(classifier_X, classifier_y, 'knn')

for m in knn_model:
    check_accuracy(m, classifier_X)

In [ ]:
gnb_model = form_model.build_tuned_model(classifier_X, classifier_y, 'gnb')

for m in gnb_model:
    print(m)
    check_accuracy(m, classifier_X)

In [ ]:
svc_model = form_model.build_tuned_model(classifier_X, classifier_y, 'svc')

for m in svc_model:
    print(m)
    check_accuracy(m, classifier_X)

In [ ]:
gmm_model = form_model.build_tuned_model(classifier_X, classifier_y, 'gmm')

for m in gmm_model:
    print(m)
    check_accuracy(m, classifier_X)

# Optimize Features

In [ ]:
def plot_RFECV(X, y):
    svc = SVC(kernel="linear")
    rfecv = RFECV(estimator=svc, step=1, cv=StratifiedKFold(y, 2),
                  scoring='accuracy')
    rfecv.fit(X, y)

    # Plot number of features VS. cross-validation scores
    plt.figure()
    plt.xlabel("Number of features selected")
    plt.ylabel("Cross validation score (nb of correct classifications)")
    plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
    plt.show()
    
plot_RFECV(classifier_X, classifier_y)

In [ ]:
def create_RFE(X, y):
    # Create the RFE object and rank each pixel
    svc = SVC(kernel="linear", C=1)
    rfe = RFE(estimator=svc, n_features_to_select=1, step=1)
    rfe.fit(X, y)
    print(rfe.ranking_)
    return rfe.ranking_

rankings = create_RFE(classifier_X, classifier_y)

In [ ]:
""" Reordering the columns in the data to reflect the rankings """
def rank_columns(data_X, rankings):
    rankings_name = []
    ranked_X = data_X

    for r in range(len(rankings)):
        column_name = data_X.iloc[:, rankings[r]-1].name
        rankings_name.append(column_name)

    ranked_X = ranked_X.reindex_axis(rankings_name, axis=1)
    
    return ranked_X

ranked_X = rank_columns(classifier_X, rankings)
display(ranked_X.head())
display(ranked_X.columns)

In [ ]:
def train_ranked_models(X, y):
    
    for x in xrange(15, 18):
        print('X :: {}'.format(x))

        ranked_models = form_model.train_models(round_number, X.iloc[:, 1:x], y, ['log',  'svc', 'gmm', 'knn', 'gnb', 'randomForest'])
        
train_ranked_models(ranked_X, classifier_y)

In [ ]:
def plot_PCA(data):
    
    pca = PCA(n_components=4)
    pca.fit(data)
    
    # Generate PCA results plot
    pca_results = rs.pca_results(data, pca)
    #print(pca_results)
    pca_data = pca.transform(data)
    
    # Create a DataFrame for the reduced data
    pca_data = pd.DataFrame(pca_data, columns = ['Dimension 1', 'Dimension 2', 'Dimension 3', 'Dimension 4'])
    #plot_data = pd.concat([preds, pca_data], axis = 1)
    
    # Scatterplotting the transformed data if it's 2 Dimensions
    """fig, ax = plt.subplots(figsize = (14,8))
    cmap = cm.get_cmap('gist_rainbow')
    
    for i, cluster in plot_data.groupby('converted_goals'):   
        cluster.plot(ax = ax, kind = 'scatter', x = 'Dimension 1', y = 'Dimension 2', 
                     color = cmap((i)*1.0/(4)), label = 'Points %i'%(i), s=30);"""
    
    return pca, pca_data, pca_results

pca, pca_data, pca_results = plot_PCA(classifier_X)

all_pca_data = pd.concat([classifier_y, pca_data], axis = 1)

print(pca_data.shape)

# Runs models on PCA to see if there is a difference in the results
pca_models = run_features(all_pca_data, [], 'converted_goals', ['log', 'knn', 'gnb', 'randomForest'])

#preds = knn.predict(pca_X)
(pca_y, pca_X) = model_libs._extract_target(all_pca_data, 'converted_goals')

for m in pca_models:
    check_accuracy(m, pca_X)

Cross Validating the SVC model with the PCA data to help prevent overfitting

In [ ]:
pca_svc_model = form_model.build_tuned_model(pca_X, pca_y, 'svc')

In [ ]:
prediction_models = form_model.load_models(['knn', 'svc', 'randomForest', 'gnb', 'gmm'])

In [ ]:
upcoming_matches, match_details = predict_matches.get_upcoming_matches()
upcoming_matches.to_csv('upcoming_matches.csv')
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 500)
upcoming_data = predict_matches.predictions(upcoming_matches)
#upcoming_data.to_csv('upcoming_data.csv')
display(upcoming_data.head())

In [ ]:
""" Need to format the data the same way we did on the raw data """
formatted_upcoming_data = upcoming_data.drop('points', 1)

formatted_upcoming_data['converted_goals'] = formatted_upcoming_data.apply(lambda row: model_libs.set_group(row['goals']), axis=1)

formatted_upcoming_data = formatted_upcoming_data.drop(ignore_cols + ['current_formation', 'goals', 'converted_goals'], 1)
display(formatted_upcoming_data.head())

In [ ]:
""" Models we'll use to predict on upcoming matches """
# pca_svc_model, knn_model, random_forest_model

# This is all the X values
formatted_upcoming_data

rf_preds = prediction_models[2].predict(formatted_upcoming_data)
print(rf_preds)
knn_preds = prediction_models[0].predict(formatted_upcoming_data)
print(knn_preds)

svc_preds = prediction_models[1].predict(formatted_upcoming_data)
print(svc_preds)

gmm_preds = prediction_models[3].predict(formatted_upcoming_data)
print(gmm_preds)

gnb_preds = prediction_models[3].predict(formatted_upcoming_data)
print(gnb_preds)

#linear_preds = lin_regr.predict(upcoming_ranked)
#print(linear_preds)


In [ ]:
columns = ['team_name', 'opp_name', 'scheduled']
# Remove all columns except the ones above
upcoming_matches = upcoming_data[columns]

# Add predictions to the end of that DF
results = pd.DataFrame({'KNN': knn_preds, 'RandomForest': rf_preds, 'SVC': svc_preds, 'GNB': gnb_preds, 'GMM': gmm_preds})
upcoming_matches = pd.concat([upcoming_matches, results], axis = 1)
reordered_matches = pd.DataFrame([])

for rows in upcoming_matches.iterrows():
    for i in upcoming_matches['team_name']:
        if rows[1]['opp_name'] == i:
            reordered_matches = reordered_matches.append(rows[1])
            reordered_matches = reordered_matches.append(upcoming_matches[upcoming_matches['team_name'].isin([i])])

reordered_matches = reordered_matches.drop_duplicates() 
columns = ['scheduled', 'team_name', 'opp_name', 'KNN', 'RandomForest', 'SVC', 'GNB', 'GMM']
reordered_matches = reordered_matches[columns]
reordered_matches.to_csv('predictions_new_model.csv')
print('Prediction CSV saved')

In [ ]:
actual_data = pd.read_csv('predictions_new_model.csv')
display(actual_data.head())
actual_data = actual_data.drop(actual_data.columns[[0]], axis=1)
actual_data['diff1'] = actual_data.apply(lambda r: model_libs.predictions_diff(r['Actual'], r['GMM']), axis=1)
accuracy = np.divide(actual_data['diff1'].sum(), float(len(actual_data['diff1'])))
print(accuracy)